A notebook for investigating the temperamental exposure behaviour.

Hypothesis is that this is a function of the frames-per-second (fps) setting, so here we will repeat a series of exposures across a range of ~0 - 5 seconds, with different FPS settings, and we will look for patterns in where the poor performance occurs.

In [14]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# Overview

- set exposures to trial
- set FPS to trial
- connect to cameras
- for each FPS
    - set FPS (for each camera)
    - for each camera
        - expose at each exposure
        - plot mean and std dev. vs. exposure time, logarithmic
        - record mean, std dev, exposure
- for each camera, 
    - overplot mean and std dev vs exposure, for all FPS superimposed.

Criteria:
- look for a straight line relationship between mean and exposure (after read noise has been exceeded)
- look for sections of poor performance, and look for responsible FPS setting.
- find the lowest allowed FPS setting for each exposure range.

# Setting the exposures

Minimum exposure: 60 µs, or 6E-5 seconds
Maximum exposure: for these purposes, set this to 5 seconds.
Let's ensure we test each power in depth. Let's use a scaling factor of 1.15 to increase exposures each time.

To get from 60 µs to 6 seconds, this requires: log(6/6E-5) / log(1.15) ~ 83 steps. This is a reasonable number of exposures to acquire.


In [13]:
import numpy as np
min_exp = 6.01E-5
max_exp = 6
exposures = min_exp * 1.14**np.arange(0, 96)
exposures

array([6.01000000e-05, 6.85140000e-05, 7.81059600e-05, 8.90407944e-05,
       1.01506506e-04, 1.15717416e-04, 1.31917855e-04, 1.50386354e-04,
       1.71440444e-04, 1.95442106e-04, 2.22804001e-04, 2.53996561e-04,
       2.89556080e-04, 3.30093931e-04, 3.76307081e-04, 4.28990073e-04,
       4.89048683e-04, 5.57515498e-04, 6.35567668e-04, 7.24547141e-04,
       8.25983741e-04, 9.41621465e-04, 1.07344847e-03, 1.22373126e-03,
       1.39505363e-03, 1.59036114e-03, 1.81301170e-03, 2.06683334e-03,
       2.35619001e-03, 2.68605661e-03, 3.06210453e-03, 3.49079917e-03,
       3.97951105e-03, 4.53664260e-03, 5.17177256e-03, 5.89582072e-03,
       6.72123562e-03, 7.66220860e-03, 8.73491781e-03, 9.95780630e-03,
       1.13518992e-02, 1.29411651e-02, 1.47529282e-02, 1.68183381e-02,
       1.91729055e-02, 2.18571122e-02, 2.49171079e-02, 2.84055030e-02,
       3.23822735e-02, 3.69157918e-02, 4.20840026e-02, 4.79757630e-02,
       5.46923698e-02, 6.23493016e-02, 7.10782038e-02, 8.10291523e-02,
      

Note I've tuned this to 96 steps, with a scale factor of 1.14.

# Connecting to cameras

In [ ]:
import orochi_sim_ctrl as osc

ic = osc.start_ic()
camera_config = osc.load_camera_config()
cameras = osc.connect_cameras(ic, camera_config)
osc.configure_cameras(cameras)

# Setting FPS values to trial

FPS values should be trialled of 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 14, 16, 18, 20, 25, 30.

However, not all of the short FPS need to be tried. Let's trials:
1, 2, 5, 10, 16, 20 , 25, 30.

In [15]:
fpss = [1, 2, 5, 10, 16, 20, 25, 30]

# Running the trials

In [ ]:
import pandas as pd

def exposure_trials(camera, fpss, exposures) -> pd.DataFrame:    
    # just caputre frame at each exposure, log the exposure actually used and the exposure commanded, and the mean and std dev of the resultant image.
    fps_dat = []
    exp_dat = []
    exp_act = []
    ave_dat = []
    std_dat = []

    for fps in fpss:
        camera.set_frame_rate(fps)
        for exposure in exposures:
            camera.set_exposure(exposure)
            img = camera.image_capture()
            img_mean = np.mean(img)
            img_std = np.std(img)
            exposure_actual = camera.get_exposure()
            fps_dat.append(fps)
            exp_dat.append(exposure)
            exp_act.append(exposure_actual)
            ave_dat.append(img_mean)
            std_dat.append(img_std)
    
    results = pd.DataFrame({'fps': fps_dat, 'exposure': exp_dat, 'exposure_actual': exp_act, 'mean': ave_dat, 'std': std_dat})
    return results

In [ ]:
import orochi_sim_proc as osp

fig_exp, ax_exp = osp.grid_plot() # init the exposure grid plot
cntr_exp = ax_exp[8] # centre plot for overlaying the exposures

In [ ]:
for camera in cameras:
    results = exposure_trials(camera, fpss, exposures)
    num = camera.number
    ax_exp[num].errorbar(results['exposure'], results['mean'], yerr=results['std'], fmt='o')
    cntr_exp.errorbar(results['exposure'], results['mean'], yerr=results['std'], fmt='o', label=f'camera {num}')

osp.show_grid(fig_exp, ax_exp)
    